In [1]:
import os
os.chdir('..')

In [2]:
import torch
import numpy as np
import copy

from util.misc import load
from model.videopose import TemporalModel
from model.pose_refinement import abs_to_hiprel, add_back_hip, gmloss, capped_l2, capped_l2_euc_err, step_zero_velocity_loss
from databases.joint_sets import MuPoTSJoints
from databases.datasets import PersonStackedMuPoTsDataset
from training.preprocess import get_postprocessor, SaveableCompose, MeanNormalize3D
from training.callbacks import TemporalMupotsEvaluator
from training.loaders import UnchunkedGenerator
from training.torch_tools import get_optimizer

LOG_PATH = "../models"

In [3]:
model_name = "29cbfa0fc1774b9cbb06a3573b7fb711"

In [4]:
def load_model(model_folder):
    config = load(os.path.join(LOG_PATH, model_folder, "config.json"))
    path = os.path.join(LOG_PATH, model_folder, "model_params.pkl")

    # Input/output size calculation is hacky
    weights = torch.load(path)
    num_in_features = weights["expand_conv.weight"].shape[1]

    m = TemporalModel(
        num_in_features,
        MuPoTSJoints.NUM_JOINTS,
        config["model"]["filter_widths"],
        dropout=config["model"]["dropout"],
        channels=config["model"]["channels"],
        layernorm=config["model"]["layernorm"],
    )

    m.cuda()
    m.load_state_dict(weights)
    m.eval()

    return config, m

In [5]:
def get_dataset(config):
    return PersonStackedMuPoTsDataset(
        config["pose2d_type"],
        config.get("pose3d_scaling", "normal"),
        pose_validity="all",
    )

In [6]:
config, model = load_model(model_name)

In [7]:
test_set = get_dataset(config)

In [8]:
refine_config = load("scripts/nn_refine_config.json")
joint_set = MuPoTSJoints()

In [9]:
def extract_post(model_name, test_set):
    params_path = os.path.join(LOG_PATH, str(model_name), "preprocess_params.pkl")
    transform = SaveableCompose.from_file(params_path, test_set, globals())
    test_set.transform = transform

    assert isinstance(transform.transforms[1].normalizer, MeanNormalize3D)
    normalizer3d = transform.transforms[1].normalizer

    return get_postprocessor(config, test_set, normalizer3d)

In [10]:
post_process_func = extract_post(model_name, test_set)

In [11]:
pad = (model.receptive_field() - 1) // 2
generator = UnchunkedGenerator(test_set, pad, True)
seqs = sorted(np.unique(test_set.index.seq))

In [15]:
seqs

['1/0',
 '1/1',
 '10/0',
 '10/1',
 '11/0',
 '11/1',
 '12/0',
 '12/1',
 '13/0',
 '13/1',
 '13/2',
 '14/0',
 '14/1',
 '14/2',
 '15/0',
 '15/1',
 '15/2',
 '16/0',
 '16/1',
 '16/2',
 '17/0',
 '17/1',
 '17/2',
 '18/0',
 '18/1',
 '18/2',
 '19/0',
 '19/1',
 '19/2',
 '2/0',
 '2/1',
 '20/0',
 '20/1',
 '20/2',
 '3/0',
 '3/1',
 '4/0',
 '4/1',
 '5/0',
 '5/1',
 '6/0',
 '6/1',
 '7/0',
 '7/1',
 '7/2',
 '8/0',
 '8/1',
 '9/0',
 '9/1']

In [12]:
refine_config['learning_rate'] = 0.0001

In [14]:
# losses = {}
optimized_preds_list = []
for i, (pose2d, valid) in enumerate(generator):
    for j in range(valid.shape[-1]):
        if (j+1) > (valid.shape[-1] - refine_config['smoothness_loss_hip_largestep']):
            reverse = True
            f = j - refine_config['smoothness_loss_hip_largestep']
            t = j + 1
        else:
            reverse = False
            f = j
            t = f + refine_config['smoothness_loss_hip_largestep'] + 1
#         print(f, t)
        model_ = copy.deepcopy(model)
        optimizer = get_optimizer(model_.parameters(), refine_config)
        for k in range(1):  # refine_config['num_iter']):
            optimizer.zero_grad()

            seq = seqs[i]
            pred3d = model_(torch.from_numpy(pose2d[:,f:t+2*pad,:]).cuda())  # [2, 401, 42] -> [2, 21+2*13, 42], pred3d: [21, 16, 3]
            valid_ = valid[0][f:t]
        #     losses[seq] = self.loss(pred3d[0][valid], self.preprocessed3d[seq]) # .cpu().numpy()

            pred_real_pose = post_process_func(pred3d[0], seq)#[f:t]  # unnormalized output

            pred_real_pose_aug = post_process_func(pred3d[1], seq)#[f:t]
            pred_real_pose_aug[:, :, 0] *= -1
            pred_real_pose_aug = test_set.pose3d_jointset.flip(pred_real_pose_aug)
            pred_real_pose = (pred_real_pose + pred_real_pose_aug) / 2


            pred = pred_real_pose[valid_]

            inds = test_set.index.seq == seq

                # for seq in seqs:
            poses_pred = abs_to_hiprel(pred, joint_set) / 1000 # (201, 17, 3)
            if k == 0:
                poses_init = poses_pred.detach().clone()
                poses_init.requires_grad = False
                kp_score = np.mean(test_set.poses2d[inds, :, 2], axis=-1)[f:t] # (201,)
                #     if refine_config['smooth_visibility']:
                #         kp_score = ndimage.median_filter(kp_score, 9)
                kp_score = torch.from_numpy(kp_score).cuda() # [201]
                scale = torch.ones((len(kp_score), 1, 1)) # torch.Size([201, 1, 1])

                kp_score.requires_grad = False
                scale.requires_grad = False


            # smoothing formulation
            if refine_config['pose_loss'] == 'gm':
                pose_loss = torch.sum((kp_score.view(-1, 1, 1) * gmloss(poses_pred - poses_init, refine_config['gm_alpha']))[0, ])
            elif refine_config['pose_loss'] == 'capped_l2':
                pose_loss = torch.sum((kp_score.view(-1, 1, 1) * capped_l2(poses_pred - poses_init, torch.tensor(refine_config['l2_cap']).float().cuda()))[0, ])
            elif refine_config['pose_loss'] == 'capped_l2_euc_err':
                pose_loss = torch.sum((kp_score.view(-1, 1) * capped_l2_euc_err(poses_pred, poses_init,
                                                                               torch.tensor(refine_config['l2_cap']).float().cuda()))[0, ])
            else:
                raise NotImplementedError('Unknown pose_loss' + refine_config['pose_loss'])

            neighbour_dist_idx = 0 if not reverse else -1
            velocity_loss_hip = globals()[refine_config['smoothness_loss_hip']](poses_pred[:, [0], :], 1)[[neighbour_dist_idx]]

            step = refine_config['smoothness_loss_hip_largestep']
            vel_loss = globals()[refine_config['smoothness_loss_hip']](poses_pred[:, [0], :], step)
            velocity_loss_hip_large = (1 - kp_score[-len(vel_loss):]) * vel_loss

            velocity_loss_rel = globals()[refine_config['smoothness_loss_rel']](poses_pred[:, 1:, :], 1)[[neighbour_dist_idx]]
            vel_loss = globals()[refine_config['smoothness_loss_rel']](poses_pred[:, 1:, :], step)
            velocity_loss_rel_large = (1 - kp_score[-len(vel_loss):]) * vel_loss

            total_loss = pose_loss + refine_config['smoothness_weight_hip'] * velocity_loss_hip \
                         + refine_config['smoothness_weight_hip_large'] * velocity_loss_hip_large \
                         + refine_config['smoothness_weight_rel'] * velocity_loss_rel \
                         + refine_config['smoothness_weight_rel_large'] * velocity_loss_rel_large
#             total_loss.backward()
#             print(total_loss.cpu().detach().numpy())

#             optimizer.step()
        optimized_preds_list.append(poses_pred[[neighbour_dist_idx]].detach().cpu().numpy())
#             break
#         break
#     break

KeyboardInterrupt: 

In [ ]:
{'total_loss': total_loss.detach().cpu(),
'pose_loss': pose_loss.detach().cpu(),
'velocity_loss_hip': velocity_loss_hip.detach().cpu(),
'velocity_loss_hip_large': velocity_loss_hip_large.detach().cpu(),
'velocity_loss_rel': velocity_loss_rel.detach().cpu(),
'velocity_loss_rel_large': velocity_loss_rel_large.detach().cpu()}

In [ ]:
poses_pred

In [ ]:
# optimized_preds_list = [o.detach().cpu().numpy() for o in optimized_preds_list]

In [ ]:
optimized_preds = np.concatenate(optimized_preds_list)

In [ ]:
pred = optimized_preds

In [ ]:
pred_gt = np.load('pre_optimize_poses.npy')

In [ ]:
pred

In [ ]:
np.save('mypred.py', optimized_preds)

In [ ]:
pred = optimized_preds # np.load('mypred.npy')

In [ ]:
from model.pose_refinement import StackedArrayAllMupotsEvaluator
from scripts.eval_nn_refine import unstack_mupots_poses

l = StackedArrayAllMupotsEvaluator(pred, test_set, True, prefix="R")
l.eval(calculate_scale_free=True, verbose=True)
exp.log_metrics(l.losses_to_log)

pred_by_seq = {}
for seq in range(1, 21):
    inds = test_set.index.seq_num == seq
    pred_by_seq[seq] = pred[inds]
pred_2d, pred_3d = unstack_mupots_poses(test_set, pred_by_seq)

In [ ]:
from databases import mupots_3d
print("\nR-PCK  R-AUC  A-PCK  A-AUC")
keys = ["R-PCK", "R-AUC", "A-PCK", "A-AUC"]
values = []
for relative in [True, False]:
    pcks, aucs = mupots_3d.eval_poses(
        False,
        relative,
        "annot3" if config["pose3d_scaling"] == "normal" else "univ_annot3",
        pred_2d,
        pred_3d,
        keep_matching=True,
    )
    pck = np.mean(list(pcks.values()))
    auc = np.mean(list(aucs.values()))
    values.append(pck)
    values.append(auc)

    print(" %4.1f   %4.1f  " % (pck, auc), end="")
print()
exp.log_metrics({f"{prefix}-{k}": v for k, v in zip(keys, values)})